# 填充数据

当数据集中含有缺失值的时候，你可以选择删除或者填充这些值。在本练习中，你要处理世界银行 GDP (Gross Domestic Product) 数据，填充缺失值。

In [ ]:
# run this code cell to read in the data set
import pandas as pd
df = pd.read_csv('../data/gdp_data.csv', skiprows=4)
df.drop('Unnamed: 62', axis=1, inplace=True)

In [ ]:
# run this code cell to see what the data looks like
df.head()

In [ ]:
# Run this code cell to check how many null values are in the data set
df.isnull().sum()

这里有不少空值。运行下方单元格中的代码将数据集里几个国家的数据绘制成图。

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# put the data set into long form instead of wide
df_melt = pd.melt(df, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], var_name='year', value_name='GDP')

# convert year to a date time
df_melt['year'] = pd.to_datetime(df_melt['year'])

def plot_results(column_name):
    # plot the results for Afghanistan, Albania, and Honduras
    fig, ax = plt.subplots(figsize=(8,6))

    df_melt[(df_melt['Country Name'] == 'Afghanistan') | 
            (df_melt['Country Name'] == 'Albania') | 
            (df_melt['Country Name'] == 'Honduras')].groupby('Country Name').plot('year', column_name, legend=True, ax=ax)
    ax.legend(labels=['Afghanistan', 'Albania', 'Honduras'])
    
plot_results('GDP')

Afghanistan 和Albania 的数据缺失了，所以这里看起来有空白。 


# 练习 - 第一部分

你的第一项任务是计算每个国家的平均 GDP，然后用平均 GDP填充缺失值。用 pandas 完成这个需要一些技巧。这是一些参考链接：
* https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.groupby.html
* https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.transform.html
* https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DataFrame.fillna.html

In [ ]:
# TODO: Use the df_melt dataframe and fill in missing values with a country's mean GDP
# If you aren't sure how to do this, 
# look up something like "how to group data and fill in nan values in pandas" in a search engine
# Put the results in a new column called 'GDP_filled'.

# HINT: You can do this with these methods: groupby(), transform(), a lambda function, fillna(), and mean()

df_melt['GDP_filled'] = None

In [ ]:
# Plot the results
plot_results('GDP_filled')

这算是提高了数据质量。至少没有了缺失值。但是因为 GDP 一般会越来越高，平均 GDP 可能并不是填充缺失值最好的选择。接下来，尝试使用向前填充 (forward fill) 的方法来处理缺失值。

# 练习 - 第二部分

使用 fillna 向前填充方法来填充缺失值。这是[文档](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DataFrame.fillna.html)。课程视频里解释过，向前填充将前一个值填到缺失值里。

pandas 的 fillna 方法有一个 forward fill 参数，例如你想要在 GDP 数据集中使用向前填充，你可以执行  `df_melt['GDP'].fillna(method='ffill')`。但是，这段代码有 2 个问题：
1. 你要先确保数据是按照年份排序的
2. 你需要按照国家名称将数据分组，这样是在每个国家内部进行向前填充。

编写代码，先将 df_melt dataframe 按照时间顺序进行排列，然后按照 'Country Name' 排序，最后向前填充。

In [ ]:
# TODO: Use forward fill to fill in missing GDP values
# HINTS: use the sort_values(), groupby(), and fillna() methods

df_melt['GDP_ffill'] = None

In [ ]:
# plot the results
plot_results('GDP_ffill')

至少对于 Afghanistan 数据，看起来好多了；但是 Albania 数据还是有缺失值。你可以使用向后填充 (back fill ) 来填充Albania 数据。这是你接下来的任务。

# 练习 - 第三部分

这部分和第二部分 (Part 2) 很像，只是这次要用向后填充了。编写代码，向后填充缺失的 GDP 数据。

In [ ]:
# TODO: Use back fill to fill in missing GDP values
# HINTS: use the sort_values(), groupby(), and fillna() methods

df_melt['GDP_bfill'] = None

In [ ]:
# plot the results
plot_results('GDP_bfill')

# 结语

这种情况下，三个国家的GDP 数据都补全了。注意，向前填充不能填充所有的 Albania 数据，因为1960年的第一行数据是 NaN。向前填充会试着填充 1961 年对应的值到 1960 年的 NaN 值上。

要完整填充每个国家的 GDP 数据，你可能得同时使用向前填充和向后填充。也要注意，先向前填充或先向后填充会导致结果微微有些差异。例如，Afghanistan 是在数据的中间部分出现了缺失值。因此，向前填充和向后填充会导致稍微不同的结果。

运行下方的代码，看看运行了向前和向后填充之后，是否能够把 GDP 数据的 NaN 值都填充满。

In [ ]:
# Run forward fill and backward fill on the GDP data
df_melt['GDP_ff_bf'] = df_melt.sort_values('year').groupby('Country Name')['GDP'].fillna(method='ffill').fillna(method='bfill')

# Check if any GDP values are null
df_melt['GDP_ff_bf'].isnull().sum()